In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [3]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../../data/train.csv'
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Do your thing crazy machine learning thing here :) ...

In [ ]:
from logistic_regression import *
from costs import *

# There are two parameters, lambda_ and gamma, where gamma is the step size 

max_iter = 15000
lambdas = np.arange(0.1, 0.4, 0.1)
gammas = np.arange(0.01, 0.1, 0.01)

# When lambda is 0, reg_logistic_regression is naive non-penalized logistic_regression.

struct = dict()
for lambda_ in lambdas:
    for gamma in gammas:
        w = reg_logistic_regression(y, tX, lambda_, gamma, max_iter)
        print("finished the regression")
        err = compute_loss(y, tX, w)
        print("finished the error")
        struct[(gamma, lambda_)] = (w, err)
        break
        
for (gamma, lambda_), (w, err) in struct.items():
    print("Gamma: ", gamma, " Lamdba: ", lambda_, " w: ", w, "error: ", err)
        
        

        


Current iteration=0, the loss=173286.79513998624


/Users/chenfs/Google Drive/EPFL/Courses/PC and ML/ML_course_origin/ML_course/projects/project1/src/python/logistic_regression.py:6: RuntimeWarning: overflow encountered in exp
  return 1.0 / (1.0 + np.exp(-t))
/Users/chenfs/Google Drive/EPFL/Courses/PC and ML/ML_course_origin/ML_course/projects/project1/src/python/logistic_regression.py:15: RuntimeWarning: divide by zero encountered in log
  prediction[i] = np.log(prediction[i]) if y[i] == 1 else np.log(1 - prediction[i])
/Users/chenfs/Google Drive/EPFL/Courses/PC and ML/ML_course_origin/ML_course/projects/project1/src/python/logistic_regression.py:45: RuntimeWarning: invalid value encountered in double_scalars
  if (loss_prev != 0) and np.abs(loss_prev - loss) < threshold:


a
## Generate predictions and save ouput in csv format for submission:

In [10]:
DATA_TEST_PATH = '../../data/test.csv' 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '../../data/output.csv' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)